In [6]:
from mica_text_coref.coref.movie_coref import rules

import jsonlines
import numpy as np
import os
from scorch import scores

In [2]:
docs = []
with jsonlines.open(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/Nov22_08:56:14PM/dev.jsonlines")) as reader:
    for doc in reader:
        docs.append(doc)

In [3]:
len(docs)

3

In [13]:
f1s = []
new_f1s = []

for doc in docs:
    print(doc["movie"])
    gold_clusters = [set((span[0], span[1]) for span in cluster) for cluster in doc["gold"]]
    pred_clusters = [set(tuple(span) for span in cluster) for cluster in doc["pred_span"]]
    gold_mentions = set(word_id for cluster in gold_clusters for word_id in cluster)
    pred_mentions = set(word_id for cluster in pred_clusters for word_id in cluster)
    print(f"{len(gold_clusters)} gold clusters, {len(pred_clusters)} pred clusters")
    print(f"{len(gold_mentions)} gold mentions, {len(pred_mentions)} pred mentions")
    f1 = 100*scores.conll2012(gold_clusters, pred_clusters)
    f1s.append(f1)
    print(f"avg F1 = {f1:.1f}")

    precision = 100*len(gold_mentions.intersection(pred_mentions))/len(pred_mentions)
    recall = 100*len(gold_mentions.intersection(pred_mentions))/len(gold_mentions)
    f1 = 2*precision*recall/(precision + recall)
    print(f"Mention Identification: P = {precision:.1f} R = {recall:.1f} F1 = {f1:.1f}")
    
    # Merge speakers
    print("merge clusters by speakers")
    pred_clusters = rules.merge_speakers(doc["token"], doc["parse"], pred_clusters)

    # Keep speakers
    print("keep speakers")
    pred_clusters = rules.keep_speakers(doc["parse"], pred_clusters)

    f1 = 100*scores.conll2012(gold_clusters, pred_clusters)
    new_f1s.append(f1)
    print(f"avg new F1 = {f1:.1f}")
    print()

print(f"average F1 = {np.mean(f1s):.1f}")
print(f"new average F1 = {np.mean(new_f1s):.1f}")

shawshank
44 gold clusters, 27 pred clusters
888 gold mentions, 885 pred mentions
avg F1 = 65.3
Mention Identification: P = 91.9 R = 91.6 F1 = 91.7
merge clusters by speakers
keep speakers
avg new F1 = 59.7

bourne
39 gold clusters, 22 pred clusters
911 gold mentions, 892 pred mentions
avg F1 = 72.6
Mention Identification: P = 92.5 R = 90.6 F1 = 91.5
merge clusters by speakers
keep speakers
avg new F1 = 67.2

basterds
23 gold clusters, 34 pred clusters
1008 gold mentions, 1055 pred mentions
avg F1 = 75.9
Mention Identification: P = 90.2 R = 94.4 F1 = 92.3
merge clusters by speakers
keep speakers
avg new F1 = 71.9

average F1 = 71.3
new average F1 = 66.2
